In [ ]:
# Load the data folder in the share drives
# The data folder is in the 'drive/Shareddrives/Ada'
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [ ]:
# installing required packages
!pip install pandas==1.0.5
!pip install textblob


     |████████████████████████████████| 10.1 MB 5.6 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.0.5 which is incompatible.


In [ ]:
# Import needed libraries:

import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import warnings
import pickle
import shutil
import os
import glob
import wordcloud
import nltk
import re
warnings.filterwarnings('ignore')
import requests 
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import sys
import spacy
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim (for LDA)
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import plotly.express as px

#URL parsing
from urllib.parse import parse_qsl, urljoin, urlparse

#Plots
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Date
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# loading NLTK Stop words

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Import Data

In [ ]:
# importing the processed/picked  data:
df=pd.read_pickle('/content/drive/Shareddrives/Ada/Uniform/All_attribute_version1312.pkl')

In [ ]:
#printing the first lines:
df.head()

,id,quoteID,quotation,speaker,probas,date,numOccurrences,urls,phase,label,date_of_birth,nationality,gender,ethnic_group,occupation,party,academic_degree,religion,polarity_TextBlob,polarity_SentimentIntensityAnalyze,polarity_Flair,polarity,emotion,day,year,just_date,domains,TLD,domain_country
0,Q20251661,2020-01-31-011304,Brexit is the victory of the common people aga...,Laura Huhtasaari,"[[Laura Huhtasaari, 0.9325], [None, 0.059], [P...",2020-01-31 03:25:04,1,[http://voanews.com/europe/shrug-and-some-sorr...,E,Laura Huhtasaari,[+1979-03-30T00:00:00Z],Finland,female,0,politician,Finns Party,0,Lutheranism,-0.022619,0.5719,0.663805,0.571900,positive,2020-01-31,2020,2020-01-31,[voanews.com],com,NaN
1,Q694676,2020-02-13-014393,Certainly Brexit was a big issue and people we...,Pat McFadden,"[[Pat McFadden, 0.9371], [None, 0.0535], [Jere...",2020-02-13 06:01:30,1,[http://www.expressandstar.com/news/politics/2...,E,Pat McFadden,[+1965-03-26T00:00:00Z],United Kingdom,male,0,politician,Labour Party,0,0,-0.161905,-0.2500,0.997470,-0.161905,negative,2020-02-13,2020,2020-02-13,[www.expressandstar.com],com,NaN
2,Q1341937,2020-03-08-010480,"I backed leave and voted leave, but I never ca...",Jason McCartney,"[[Jason McCartney, 0.8274], [None, 0.128], [Li...",2020-03-08 05:45:58,1,[http://www.yorkshirepost.co.uk/news/second-ti...,E,Jason McCartney,[+1973-09-03T00:00:00Z],United States of America,male,0,sport cyclist,0,0,0,0.000000,-0.4243,0.595703,0.000000,negative,2020-03-08,2020,2020-03-08,[www.yorkshirepost.co.uk],uk,United Kingdom
3,Q6837191,2020-02-14-058019,"legacy, bad faith on pensions, Brexit and poli...",Michelle O'Neill,"[[Michelle O'Neill, 0.8055], [None, 0.1302], [...",2020-02-14 11:48:49,1,[https://www.belfasttelegraph.co.uk/news/north...,E,Michelle O'Neill,[+1977-01-10T00:00:00Z],Northern Ireland,female,0,politician,Sinn Féin,0,0,-0.233333,-0.0516,0.951277,-0.051600,negative,2020-02-14,2020,2020-02-14,[www.belfasttelegraph.co.uk],uk,United Kingdom
4,Q1728820,2020-01-27-094916,"we're only at half-time on Brexit, it's not do...",Leo Varadkar,"[[Leo Varadkar, 0.9378], [None, 0.0502], [Bori...",2020-01-27 12:25:00,1,[http://blogs.independent.co.uk/voices/brexit-...,E,Leo Varadkar,[+1979-01-18T00:00:00Z],Ireland,male,0,politician,Fine Gael,0,0,0.000000,0.0000,0.946059,0.000000,negative,2020-01-27,2020,2020-01-27,[blogs.independent.co.uk],uk,United Kingdom


In [ ]:
# printing all the columns:
df.columns

Index(['id', 'quoteID', 'quotation', 'speaker', 'probas', 'date',
       'numOccurrences', 'urls', 'phase', 'label', 'date_of_birth',
       'nationality', 'gender', 'ethnic_group', 'occupation', 'party',
       'academic_degree', 'religion', 'polarity_TextBlob',
       'polarity_SentimentIntensityAnalyze', 'polarity_Flair', 'polarity',
       'emotion', 'day', 'year', 'just_date', 'domains', 'TLD',
       'domain_country'],
      dtype='object')

#DATA STORY/ Pipeline:
**I - The media's coverage of Brexit from 2015 to 2020 considering data in Quotebank:**


1. How many quotations are related to Brexit?
2. When did the media start to mention Brexit? 
3. How did the number of quotations change over time?

  
**II - Media views towards Brexit:**


1. Which media have relayed the information the most?
2. What is the country of origin of these media?
3. What media and websites tend to have positive or negative statements towards the situation?
4. How sentiments of quotations revealed by top 10 medias change over recent years? 


**III - Who issued these quotes, and what were the main opinions:**


1.  Plot: Word cloud of the quotation :
2. How many active speakers are mentioned?
3. Most active speaker with different Sentiment towards Brexit
4. What are the main topics discussed
5. Where do they come from?
6.  How has the number of active speakers changed over the different periods of Brexit? 
7. What is their sentiment about the situation, content or not content, independently of their opinion on pro or against Brexit?
8. Did they change their perception during this period?
9. How are the different sentiments of the speakers distributed by countries?


# Methods: 

* To merge quotebank with the additional datasets, we have filtered those quotations related to Brexit by identifying possible keywords related to Brexit. We have already combined quotations with speaker attributes by joining with QIDs.  We also scrapped [the web page](https://icannwiki.org/Country_code_top-level_domain) to get the domain countries using beautiful soup for future analysis.


* We then carried out a sentiment analysis, for that we used the results of three packages: Textblob, SentimentIntensityAnalyze and Flair. We have then taken the median of the three resulting polarities. To subsequently decide on the thresholds to choose for classifying the emotions (Positive, Negative, or Neutral), we manually labeled 100 quotes and selected the thresholds accordingly.  It is critical to note that the resulting emotion does not reflect the direct view or perception of the Brexit, i.e. pro or con. The resulting emotion is based on the tone of the quote. As an example (fabricated), the two quotes: "Brexit is crucial, staying in the union would be a catastrophe, a suicide" and "Staying in the union is crucial, leaving would be a catastrophe, a suicide" will both be assigned a negative emotion by design even though they stand for two different opinions.


* We also performed an LDA topic modeling to get an insight about the most relevant subjects. This approach allowed us to notice clearly the different names involved with Brexit like Theresa May, Boris Johnson, Donald Trump, David Cameron etc..

  Also, this analysis revealed some major keywords that allowed us to get some ideas about what was beeing discussed.

   We can see that some of the speakers are projecting the Brexit as an opportunity for a better change, and some of them are reflecting all the risks that may accure, stating words like (risk, damage, lose, crisis, impact etc..).
And also, we can see how some keywords are reflecting the will the stay in europe (partners, stay, deal, relationship, members etc).

  But, this was not enough for us since we thout it would be more interesting to see the position of the most involved speakers with Brexit.

  That's why we looked at the the most active ones and divided our research into 3 different timelines:
  - just before the referendum (before 2016-06-20)
  - period of referendum (from 2016-06-21 to 2016-06-25) 
  - and just after (after 2016-06-26)

  And at the same time, we were interested in answering these questions:

  1) From which countries are speakers the most interested in Brexit?

  2) Who are the most relevant speakers?

  3) What's their opinion during those different periods?


  The distribution of quotations by speakers' nationality confirmed our hypothesis: countries with the most interest with United Kingdom are the most involved with the Brexit situation. 

  We also distinguished the main speakers by their activity throughout the Brexit period of time.
  

  Our analysis just allowed ton conclude more about the position of the most relevant speakers at that time.

  


* Spectral clustering was applied for speakers and for different subclasses of occupation over time.  We first encoded the category features-nationality, polarity, and occupation-using two methods: hot encoding and target encoding. We then used the T-SNE method to compress the features and generate tsne_x,tesn_y so that they could be plotted in 2D or 3D graphs. We obtained some plots and results, but we will leave the final interpretation and more advanced implementation of the T-SNE approach to be done as a future improvement of our T-SNE method. You can find the first results, uncommented, of the cluster analysis at the end of this notebook. 

## II - Media views towards Brexit:

**Note:** Throughout this report, we will use both the terms domain and media interchangeably to refer to the source outlets. 

### II.1 Which media have relayed the information the most? 
First, we look at how many unique media we have. 

In [ ]:
print("The number of unique domains is ",df['domains'].apply(lambda x: x[0]).nunique())

The number of unique domains is  3990


In [ ]:
# From nested list to text
df['domains']=df['domains'].apply(lambda x: x[0])

In [ ]:
# group by id and do the counts to plot later:
df_domains = df.groupby(df['domains'])['id'].count().reset_index().rename(columns={'id':'counts'}).sort_values(['counts'],ascending=False)

In [ ]:
# top 10 domains
df_domains.head(10)

,domains,counts
484,express.co.uk,3819
1895,www.belfasttelegraph.co.uk,3147
674,heraldscotland.com,2275
3270,www.politico.eu,1976
3037,www.msn.com,1960
1829,www.aol.co.uk,1755
1958,www.breitbart.com,1546
3272,www.politicshome.com,1525
738,inews.co.uk,1500
2995,www.mirror.co.uk,1389


In [ ]:
# Visualise the top 20 media which relayed the information the most
fig = px.bar(df_domains.head(20),x='counts', y='domains', orientation='h')
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})

fig.show()

fig.write_html("/content/drive/Shareddrives/Ada/pickle/top20_media.html")

Among the top medias we see:

* The Daily Express (here Express) and its sister paper - the Sunday Express are daily national middle-market and conservative tabloid newspapers in the United Kingdom. 
* The Belfast Telegraph is a daily newspaper published in Belfast, Northern Ireland, by Independent News & Media. 
* The Herald is a Scottish broadsheet newspaper founded in 1783. The Herald is the longest running national newspaper in the world and is the eighth oldest daily paper in the world.
* The Politico is a political journalism company based in Arlington County, Virginia, that covers politics and policy in the United States and internationally. It primarily distributes content online but also with printed newspapers, radio, and podcasts. 
* MSN.com is a visited portal website provided by Microsoft. 

We are willing to investigate their sentiments and the pitch in which they relayed the news about the Brexit.

### II.2 What is the country of origin of these media?

We now look at the country of origin of the media. 

In [ ]:
# Remove null values in countries
domain_country_df=df[df['domain_country'].notnull()].reset_index()
# Read 3-digits ISO code for 
country_code_df=pd.read_csv('/content/drive/Shareddrives/Ada/countries_codes.csv')
country_code_df=country_code_df[['country','id']]
country_code_df=country_code_df.rename(columns={'id':'country_code'})

In [ ]:
# count the country domains
count_domain_countries=domain_country_df.groupby(['domain_country'])['domains'].nunique().reset_index()
count_domain_countries=count_domain_countries.rename(columns={'domain_country': 'country','domains': 'number of medias'}).sort_values(['number of medias'],ascending=False)

Then considering only the number of quotations in UK domain:

In [ ]:
freq_uk = 0
for index, row in df_domains.iterrows():
  if row['domains'].endswith('uk'):
    freq_uk += row['counts']

freq_uk

print('The percentage of the number of quotations in UK domains compared total: {:.2%}'.format(freq_uk/total_quotes))

the percentage of the number of quotations in UK domains compared total: 36.18%


**Note**: to decide to which country each domain is assigned, we used the link that can be not always reliable since some domains can use .com.

Now considering the medias worldwide and plotting them on an interactive map:

In [ ]:
# preparing data for pyplot:
plot_domain_country=pd.merge(left=count_domain_countries, right=country_code_df, how='left', on='country')

In [ ]:
# Since it's not possible to include the whole european union on map, we pick france as the representative country in European Union to be shown in the futher plot
plot_domain_country.at[8,'country_code']='FRA'
plot_domain_country.loc[8]

country             European Union
number of medias                14
country_code                   FRA
Name: 8, dtype: object

In [ ]:
# map plot
fig = px.scatter_geo(plot_domain_country, locations="country_code", color="country",
                     hover_name="country", size="number of medias",
                     projection="natural earth")
fig.show()

As we can see in the map above, the majority of the media concentration is in the UK. In second place we find, Europe in wide and several Commun Wealth countries like Australia, Canada, South Africa and New Zealand.

### II.3 What media and websites tend to have positive or negative statements towards the situation? 

In [ ]:
# Here we care about the domain and their emotion
df=df[['domains','emotion']]
df['domains'] = df['domains'].map(lambda a: a[0])

# Filter out the "www." and ".com"
words=["www.",".com"]
for word in words:
     df['domains'] = df['domains'].str.replace(word, ' ')

# Get top K datasets based on feature in dataframe
def top_k_feature(X,feature,k):
  X_=pd.DataFrame()
  a=X[feature].value_counts()[:k].index.tolist()
  X_=X.loc[X[feature].isin(a)]
  return X_

# Get top 20 feature
df=top_k_feature(df,'domains',20)
df.head(5)

,domains,emotion
3,belfasttelegraph.co.uk,negative
5,breitbart,negative
9,inews.co.uk,positive
15,independent.co.uk,neutral
17,politico.eu,positive


In [ ]:
# Group by domains
group = df.groupby('domains')

# Lists contain attitudes value
keys=[]
positive=[]
negative=[]
neutral=[]

# Count all the attitude during each groups
for key, df in group:
    keys.append(key)
    positive.append(df['emotion'].value_counts()['positive'])
    neutral.append(df['emotion'].value_counts()['neutral'])
    negative.append(df['emotion'].value_counts()['negative'])
    
# Valued array keep all the emotion counts
valued=[]
valued.extend(positive)
valued.extend(negative)
valued.extend(neutral)

# k is the top K features
k=20

# Construct the target list for the sankey diagram 
targetd=[]
for i in range(0,k):
    targetd.append(0)
for i in range(0,k):
    targetd.append(1)
for i in range(0,k):
    targetd.append(2)

# construct the source list for the sankey diagram
sourced=[]
for j in range(0,3):
  for i in range(3,3+k):
      sourced.append(i)

print("source",sourced)
print("target",targetd)
print("values",valued)

source [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
target [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
values [475, 303, 863, 368, 339, 298, 372, 375, 319, 477, 456, 441, 217, 213, 319, 268, 910, 515, 405, 229, 863, 725, 1527, 784, 631, 616, 640, 669, 739, 1007, 1024, 761, 506, 493, 681, 581, 1888, 1189, 756, 514, 417, 343, 757, 394, 323, 295, 355, 327, 331, 476, 496, 323, 246, 207, 316, 310, 1021, 571, 339, 278]


In [ ]:
# plot: 
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 10,
      thickness = 50,
      line = dict(color = "black", width = 0.5),
      # Here is the label we used
      label = ["Positive", "Negative","Neutral",' aol.co.uk', ' bbc.co.uk', ' belfasttelegraph.co.uk', ' breitbart ', ' express.co.uk', ' independent.co.uk', ' independent.ie', ' irishtimes ', ' mirror.co.uk', ' msn ', ' politico.eu', ' politicshome ', ' scotsman ', ' standard.co.uk', ' thenational.scot', ' thesun.co.uk', 'express.co.uk', 'heraldscotland ', 'inews.co.uk', 'uk.investing '],
      color = ["rgba(31, 119, 180, 0.8)",
                    "rgba(255, 127, 14, 0.8)",
                    "rgba(44, 160, 44, 0.8)",
                    "rgba(214, 39, 40, 0.8)",
                    "rgba(148, 103, 189, 0.8)",
                    "rgba(140, 86, 75, 0.8)",
                    "rgba(227, 119, 194, 0.8)",
                    "rgba(127, 127, 127, 0.8)",
                    "rgba(188, 189, 34, 0.8)",
                    "rgba(23, 190, 207, 0.8)",
                    "rgba(31, 119, 180, 0.8)",
                    "rgba(255, 127, 14, 0.8)",
                    "rgba(44, 160, 44, 0.8)",
                    "rgba(214, 39, 40, 0.8)",
                    "rgba(148, 103, 189, 0.8)",
                    "rgba(140, 86, 75, 0.8)",
                    "rgba(227, 119, 194, 0.8)",
                    "rgba(127, 127, 127, 0.8)",
                    "rgba(188, 189, 34, 0.8)",
                    "rgba(23, 190, 207, 0.8)",
                    "rgba(31, 119, 180, 0.8)",
                    "rgba(255, 127, 14, 0.8)",
                    "rgba(44, 160, 44, 0.8)",
      ]
    ),
    link = dict(
      source=sourced,
      target=targetd,
      value=valued

  ))])

fig.update_layout(title_text="Emotions of top 20 media and websites", font_size=10)

fig.update_layout(
    height=900)
fig.show()

We observe that, considering the top 20 media outlets present in Quotebank's data, most of them have a negative sentiment towards the Brexit, regardless of their views. This is noted particularly with the Daily Express (here express.co.uk) which is a very conservative tabloid in the UK. 
It is surprising to note that most of the opinions are negative despite the fact that the Brexit was voted for yes. Again, we must keep in mind that this does not reflect the opinion but rather the tone of the quote.

### II.4 How sentiments of quotations revealed by top 10 medias change over recent years?

Out the top 20 we picked the top 10 to see how their emotions changed with time:

In [ ]:
# filtering the top 10
top10_media=df_domains['domains'].head(10).tolist()
# taking the columns we are interested in
df_top10_media=df[df['domains'].isin(top10_media)].groupby(['domains','year','emotion'])['id'].count().reset_index()
# doing the count
df_top10_media=df_top10_media.rename(columns={'id': 'yearly_counts'})

# removing the redundant words
base_list = ['www.','.com','.uk','.co']

#final dataset to plot
df_top10_media['media']=df_top10_media['domains'].str.replace("|".join(base_list),'')

In [ ]:
# small view of the data
df_top10_media

,domains,year,emotion,yearly_counts,media
0,express.co.uk,2015,negative,1,express
1,express.co.uk,2015,neutral,1,express
2,express.co.uk,2015,positive,2,express
3,express.co.uk,2016,negative,91,express
4,express.co.uk,2016,neutral,46,express
...,...,...,...,...,...
152,www.politicshome.com,2019,neutral,151,politicshome
153,www.politicshome.com,2019,positive,198,politicshome
154,www.politicshome.com,2020,negative,22,politicshome
155,www.politicshome.com,2020,neutral,8,politicshome


In [ ]:
# plot:
fig = px.bar(df_top10_media, x="year", y="yearly_counts", color="emotion", barmode="stack",
             facet_col="media", facet_col_wrap=5,
             category_orders={"media": top10_media})
fig.update_traces(marker_line_width=0)
fig.show()


 We can see that, in general, the proportion between negative, positive and neutral feelings has remained constant over the years for the top ten. There do not seem to be any apparent trends in the development of polarities over time. 
We can, however, notice again that the number of citations in 2020 always drops drastically.
